In [0]:
dbutils.fs.ls("mnt/silverhdata/")

[FileInfo(path='dbfs:/mnt/silverhdata/2017-2024/', name='2017-2024/', size=0, modificationTime=1712345904000),
 FileInfo(path='dbfs:/mnt/silverhdata/2017-2024.csv/', name='2017-2024.csv/', size=0, modificationTime=1712340469000),
 FileInfo(path='dbfs:/mnt/silverhdata/2024/', name='2024/', size=0, modificationTime=1712345912000),
 FileInfo(path='dbfs:/mnt/silverhdata/2024.csv/', name='2024.csv/', size=0, modificationTime=1712340469000)]

In [0]:
# get data from silver container
data = "/mnt/silverhdata/2024/"
data1 =  "/mnt/silverhdata/2017-2024/"

df = spark.read.format("delta").load(data)
df1 = spark.read.format("delta").load(data1)
df.limit(10).display()
df1.limit(10).display()

country_name,year,score,gdp,social_support,healthy_life_expectancy,freedom_rights,generosity,corruption
Afghanistan,2023,1.446,null,0.368,55.2,0.228,null,0.738
Albania,2023,5.445,9.689,0.691,69.2,0.872,0.068,0.855
Argentina,2023,6.393,9.994,0.892,67.3,0.832,-0.129,0.846
Armenia,2023,5.679,9.73,0.819,68.2,0.819,-0.179,0.681
Australia,2023,7.025,10.846,0.896,71.2,0.876,0.187,0.482
Austria,2023,6.636,10.93,0.874,71.4,0.874,0.209,0.529
Azerbaijan,2023,5.214,9.637,0.713,64.1,0.829,-0.16,0.627
Bahrain,2023,5.959,10.877,0.817,65.6,0.869,0.155,null
Bangladesh,2023,4.114,8.783,0.45,64.8,0.919,0.019,0.756
Belgium,2023,6.944,10.883,0.896,71.2,0.87,0.065,0.522


country_name,year,score,gdp,social_support,healthy_life_expectancy,freedom_rights,generosity,corruption
Afghanistan,2008,3.724,7.35,0.451,50.5,0.718,0.164,0.882
Afghanistan,2009,4.402,7.509,0.552,50.8,0.679,0.187,0.85
Afghanistan,2010,4.758,7.614,0.539,51.1,0.6,0.118,0.707
Afghanistan,2011,3.832,7.581,0.521,51.4,0.496,0.16,0.731
Afghanistan,2012,3.783,7.661,0.521,51.7,0.531,0.234,0.776
Afghanistan,2013,3.572,7.68,0.484,52.0,0.578,0.059,0.823
Afghanistan,2014,3.131,7.671,0.526,52.3,0.509,0.102,0.871
Afghanistan,2015,3.983,7.654,0.529,52.6,0.389,0.078,0.881
Afghanistan,2016,4.22,7.65,0.559,52.925,0.523,0.04,0.793
Afghanistan,2017,2.662,7.648,0.491,53.25,0.427,-0.123,0.954


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# sort by score in descending order
df = df.orderBy(F.col("score").desc())

# add new column with ranks based on score
df = df.withColumn("rank", F.rank().over(Window.orderBy(F.col("score").desc())))

df.limit(10).display()

country_name,year,score,gdp,social_support,healthy_life_expectancy,freedom_rights,generosity,corruption,rank
Finland,2023,7.699,10.808,0.947,71.3,0.943,-0.001,0.185,1
Iceland,2023,7.562,10.934,0.979,72.1,0.918,0.299,0.697,2
Denmark,2023,7.504,10.996,0.916,71.5,0.923,0.089,0.184,3
Costa Rica,2023,7.384,10.021,0.875,70.0,0.933,-0.067,0.767,4
Netherlands,2023,7.255,10.977,0.915,71.7,0.847,0.223,0.424,5
Norway,2023,7.249,11.125,0.952,71.6,0.938,0.219,0.245,6
Sweden,2023,7.161,10.902,0.927,72.4,0.926,0.147,0.253,7
Kuwait,2023,7.13,10.812,0.89,70.0,0.898,0.136,null,8
Australia,2023,7.025,10.846,0.896,71.2,0.876,0.187,0.482,9
Luxembourg,2023,7.016,11.649,0.879,71.7,0.911,0.033,0.343,10


In [0]:
# reorder
df = df.select("country_name",  "year", "score", "rank","gdp", "social_support", "healthy_life_expectancy", "freedom_rights", "generosity", "corruption")
df.limit(10).display()

country_name,year,score,rank,gdp,social_support,healthy_life_expectancy,freedom_rights,generosity,corruption
Finland,2023,7.699,1,10.808,0.947,71.3,0.943,-0.001,0.185
Iceland,2023,7.562,2,10.934,0.979,72.1,0.918,0.299,0.697
Denmark,2023,7.504,3,10.996,0.916,71.5,0.923,0.089,0.184
Costa Rica,2023,7.384,4,10.021,0.875,70.0,0.933,-0.067,0.767
Netherlands,2023,7.255,5,10.977,0.915,71.7,0.847,0.223,0.424
Norway,2023,7.249,6,11.125,0.952,71.6,0.938,0.219,0.245
Sweden,2023,7.161,7,10.902,0.927,72.4,0.926,0.147,0.253
Kuwait,2023,7.13,8,10.812,0.89,70.0,0.898,0.136,null
Australia,2023,7.025,9,10.846,0.896,71.2,0.876,0.187,0.482
Luxembourg,2023,7.016,10,11.649,0.879,71.7,0.911,0.033,0.343


In [0]:
# Drop columns in df1 with year less than 2017
df1 = df1.filter("year >= 2017")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#specify window
w = Window.partitionBy('year').orderBy(F.col("score").desc())

#add column called 'rank' that contains rank numbers
df1 = df1.withColumn('rank', F.rank().over(w))

#view DataFrame
df1.limit(10).display()

country_name,year,score,gdp,social_support,healthy_life_expectancy,freedom_rights,generosity,corruption,rank
Finland,2017,7.788,10.77,0.964,70.85,0.962,-0.008,0.192,1
Denmark,2017,7.594,10.922,0.952,70.75,0.955,0.149,0.181,2
Norway,2017,7.579,11.076,0.95,71.3,0.953,0.228,0.25,3
Iceland,2017,7.476,10.927,0.967,71.95,0.939,0.24,0.727,4
Switzerland,2017,7.474,11.13,0.95,72.1,0.925,0.173,0.316,5
Netherlands,2017,7.459,10.917,0.937,71.25,0.92,0.245,0.363,6
Canada,2017,7.415,10.786,0.934,71.2,0.945,0.157,0.362,7
Israel,2017,7.331,10.583,0.916,72.1,0.768,0.138,0.793,8
New Zealand,2017,7.327,10.65,0.955,70.05,0.942,0.287,0.222,9
Austria,2017,7.294,10.9,0.906,70.65,0.89,0.128,0.518,10


In [0]:
# reorder the columns
df1 = df1.select("country_name",  "year", "score", "rank","gdp", "social_support", "healthy_life_expectancy", "freedom_rights", "generosity", "corruption")


In [0]:
# Write df to datalakestorage in delta format
df.write.mode("overwrite").option("header", 'true').format("delta").save("/mnt/goldhdata/2024/")

# Write df_2023 to datalakestorage in delta format
df1.write.mode("overwrite").option("header", 'true').format("delta").save("/mnt/goldhdata/2017-2024/")